In [1]:
import numpy as np
from fairseq.models.transformer_lm import TransformerLanguageModel
import torch
torch.cuda.set_device(2)
torch.cuda.current_device()

2

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# path = "/home/usuaris/scratch/belen.alastruey"
# custom_lm = TransformerLanguageModel.from_pretrained(path, 'checkpoint_best.pt', bpe='sentencepiece', sentencepiece_model=path + "/en.model")
# custom_lm.score('ladies and gentlemen , i would like to begin by thanking the rapporteur , mr graça moura , for his report .', beam=5)
# custom_lm.sample('ladies and gentlemen', beam=5)

In [4]:
from fairseq_transformer_wrapper import FairseqTransformerHub, parse_single_alignment
import warnings
import torch

In [5]:
europarl_dir = "/home/usuaris/veu/javier.ferrando/norm-analysis-of-transformer/exp2_nmt/work/"
ckpt_dir = "/home/usuaris/scratch/belen.alastruey/europarl_lm_ckpt"
#ckpt_dir = "/home/usuaris/scratch/belen.alastruey"
#dict_dir = "/home/usuaris/veu/belen.alastruey/europarl_lm_data/data-bin"
dict_dir = "/home/usuaris/scratch/belen.alastruey/data-bin"

### Alignment test (BPE from Europarl)
test_de_bpe = europarl_dir + "processed_data/test.bpe.de"
test_de_word =  europarl_dir + "processed_data/test.de"
test_en_bpe = europarl_dir + "processed_data/test.bpe.en"
test_en_word = europarl_dir + "processed_data/test.en"
gold_alignment = europarl_dir + "gold_alignment/alignment.talp"

warnings.simplefilter('ignore')

with open(test_de_bpe, encoding="utf-8") as fbpe:
    src_bpe_sents = fbpe.readlines()
with open(test_en_bpe, encoding="utf-8") as fbpe:
    tgt_bpe_sents = fbpe.readlines()
with open(europarl_dir + "data_in_progress/test.uc.de", encoding="utf-8") as fword:
    src_word_sents = fword.readlines()
with open(europarl_dir + "data_in_progress/test.uc.en", encoding="utf-8") as fword:
    tgt_word_sents = fword.readlines()


hub = FairseqTransformerHub.from_pretrained(
    ckpt_dir,
    checkpoint_file=f"checkpoint_best.pt",
    data_name_or_path=(dict_dir),
    #(data_name_or_path=iwslt14_dir / "data-bin").as_posix(),
)

2022-02-09 18:31:01 | INFO | fairseq.file_utils | loading archive file /home/usuaris/scratch/belen.alastruey/europarl_lm_ckpt
2022-02-09 18:31:01 | INFO | fairseq.file_utils | loading archive file /home/usuaris/scratch/belen.alastruey/data-bin
2022-02-09 18:31:01 | INFO | fairseq.tasks.language_modeling | dictionary: 10001 types
2022-02-09 18:31:02 | INFO | fairseq.models.fairseq_model | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'tensorboard_logdir': None, 'wandb_project': 'lm', 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scal

In [6]:
# Test data
i = 0 # 0, 1 28, 45, 148 (pred_tok vs tgt_tok), 178, 16 (Hallucination)

src_sent, src_tok, src_tensor = hub.get_lm_sample('test', i)

2022-02-09 18:31:04 | INFO | fairseq.data.data_utils | loaded 508 examples from: /home/usuaris/scratch/belen.alastruey/data-bin/test


In [7]:
src_sent

'we do not believe that we should cherry-pick .but this is not what happens .of course , if a drug addict becomes a pusher , then it is right and necessary that he should pay and answer before the law also .commissioner , ladies and gentlemen , i should like to begin by thanking mr burtone for his report .<unk> legal drugs <unk> ( tranquillizers ) finding their way on to an illegal market , especially when used in combination with alcohol , are a major and serious problem particularly for young people .i should like to take this opportunity to express my thanks to mr titley .it was a very difficult position and i therefore am delighted about the new agreement .we must accept that , in the realm of blood and plasma collection , the systems vary and operate on mixed bases .the minutes of yesterday <unk>s sitting have been distributed .to this end the oostlander report takes substantial note of the european commission guidelines which are very clear , very precise and do not leave room fo

In [8]:
src_tensor.size()

torch.Size([512])

In [9]:
# src_tensor = src_tensor.unsqueeze(0).to(device)
# tgt_tensor = torch.cat([
#     torch.tensor([self.task.tgt_dict.eos_index]),
#     tgt_tensor[:-1]
#     ]).unsqueeze(0).to(self.device)

# model_output, encoder_out = self.models[0](src_tensor, src_tensor.size(-1), tgt_tensor, )

# log_probs = self.models[0].get_normalized_probs(model_output, log_probs=True, sample=None)

In [12]:
model_output, log_probs, encoder_out, layer_inputs, layer_outputs = hub.trace_lm_forward(src_tensor)

In [13]:
probs = torch.nn.functional.softmax(log_probs, dim=-1)
print('probs',probs.size())
pred_ind = torch.argmax(probs,dim=-1)
print(pred_ind.size())
pred = torch.max(probs,dim=-1)




probs torch.Size([512, 10001])
torch.Size([512])


In [14]:
pred

torch.return_types.max(
values=tensor([0.1324, 0.1331, 0.9158, 0.2433, 0.7275, 0.2566, 0.3067, 0.1303, 0.6539,
        0.8635, 0.2156, 0.1991, 0.6005, 0.7885, 0.9541, 0.3908, 0.3348, 0.4230,
        0.2484, 0.2070, 0.5856, 0.3318, 0.9750, 0.2948, 0.9861, 0.4293, 0.3794,
        0.5040, 0.1245, 0.3957, 0.9941, 0.3281, 0.4382, 0.2849, 0.6068, 0.5862,
        0.2288, 0.2559, 0.4527, 0.1248, 0.4309, 0.8498, 0.3810, 0.4883, 0.6681,
        0.3132, 0.0995, 0.1433, 0.2859, 0.2204, 0.2984, 0.4929, 0.1873, 0.9922,
        0.1302, 0.9389, 0.2174, 0.9997, 0.9999, 0.9888, 0.3022, 0.1779, 0.7726,
        0.9213, 0.1495, 0.9308, 0.3320, 0.4062, 0.0810, 0.8342, 0.9776, 0.8506,
        0.6273, 0.3768, 0.3687, 0.9780, 0.2071, 0.0459, 0.1334, 0.4097, 0.6145,
        0.3618, 0.1248, 0.1340, 0.5406, 0.8734, 0.8165, 0.1584, 0.6659, 0.4643,
        0.3473, 0.6764, 0.5239, 0.5841, 0.4260, 0.1474, 0.1369, 0.4098, 0.1238,
        0.2507, 0.2827, 0.2580, 0.2420, 0.9594, 0.2201, 0.3300, 0.1661, 0.0801,
        0